# Creating Expectations on 

As your data products and models are developed, you can encode assumptions about input and output datasets as **expectations**.

Using that workflow provides the following benefits:

1. These are machine verifiable and can be used to monitor data flowing through your pipelines.
2. These eliminate poisonous implicit assumptions that cause data engineers re-work and waste time - "How do we define visits?"
3. These **will eventually** be easy to edit.
4. These **will eventually** be easy to reason about visually.

In [1]:
import json
import os

import great_expectations as ge
import pandas as pd

Unable to load spark context; install optional spark dependency for support.
Unable to load spark context; install optional spark dependency for support.


## Initialize a DataContext

A great expectations `DataContext` represents the collection of data asset specifications in this project.

You'll need:
- the directory where you ran `great_expectations init` (where the .great_expectations.yml file is).
- dbt profile and target information in the datasources section of your great_expectations configuration

In [2]:
#context = ge.data_context.DataContext('../../', expectation_explorer=True)
context = ge.data_context.DataContext('../../', expectation_explorer=False)

## Get a Dataset

Using the data context, provide the name of the datasource configured in your project config ("dbt" in this case), and the name of the dbt model to which to connect

In [8]:
df = context.get_data_asset("mycsvfile", data_asset_name="titanic_input_file", file_path="tutorial_data/Titanic.csv")

In [ ]:
df.get_expectations_config()

## Declare Expectations

As you develop your code

state an assumption your code makes about its input data

Check on the available data sample if you can expect this assumption to be true

If the available data sample violates this assumption, decide how your code should deal with the violations

Update your assumption

In [ ]:
df.head(10)

### Can we assume that "male" and "female" are the only values we will see in "Sex" column?

In [ ]:
df.expect_column_values_to_be_in_set('Sex', ['female', 'male'], include_config=True)

### Yes. Let's keep this expectation - if our code encounters input data that does not conform to it, we want to know about

### Can we assume that all people in our input data have non-empty value in "Age" column?

In [5]:
df.expect_column_values_to_not_be_null('Age')

{'success': False,
 'result': {'element_count': 1313,
  'missing_count': 0,
  'missing_percent': 0.0,
  'unexpected_count': 557,
  'unexpected_percent': 0.4242193450114242,
  'partial_unexpected_list': []}}

### No. We will have to adjust our code to deal with nulls in this column. However, let's make sure that if in future our code encounters input data where there more nulls than we expect, we will be notified:

In [6]:
df.expect_column_values_to_not_be_null('Age', mostly=0.5)

{'success': True,
 'result': {'element_count': 1313,
  'missing_count': 0,
  'missing_percent': 0.0,
  'unexpected_count': 557,
  'unexpected_percent': 0.4242193450114242,
  'partial_unexpected_list': []}}

### Can we assume that all "Age" column values are in a reasonable range?

In [ ]:
df.expect_column_value_lengths_to_be_between('Age', min_value=0, max_value=120)

### Yes. Great - let's keep this expectation. Our code can assume that this is true. If in future we will see input data that violates this expectation, our validation will catch it.

### Let's review the expectations.

In [7]:
df.get_expectations_config()

	1 failing expectations
	1 result_format kwargs
	0 include_configs kwargs
	0 catch_exceptions kwargs
If you wish to change this behavior, please set discard_failed_expectations, discard_result_format_kwargs, discard_include_configs_kwargs, and discard_catch_exceptions_kwargs appropirately.


{'data_asset_name': 'None',
 'meta': {'great_expectations.__version__': '0.6.0__develop__sch_internal'},
 'expectations': [{'expectation_type': 'expect_column_values_to_not_be_null',
   'kwargs': {'column': 'Age', 'mostly': 0.5}}],
 'data_asset_type': 'Dataset'}

### and save them. Expectations for "titanic_input_file" will be saved in a JSON file in great_expectations/data_asset_configurations directory. We will load this file when we need to validate.

In [ ]:
df_every_visit_per_day.save_expectations_config()